In [1]:
import numpy as np
import json, random, os, copy, time
from pprint import pprint
import pickle
from collections import Counter, defaultdict
from nltk.corpus import stopwords

In [6]:
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0823_clean_te.json", "r"))
img_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0823_clean_te.json", "r"))

print(Counter([txt_dataset[k]['split'] for k in txt_dataset]))
print(len(set([txt_dataset[k]['Guid'] for k in txt_dataset])))

print(Counter([img_dataset[k]['split'] for k in img_dataset]))
print(Counter([img_dataset[k]['Qcate'] for k in img_dataset]))
print(len(set([img_dataset[k]['Guid'] for k in img_dataset])))

Counter({'train': 17812, 'test': 4076, 'val': 2455})
24343
Counter({'train': 16448, 'test': 3464, 'val': 2511})
Counter({'YesNo': 7430, 'Others': 5823, 'choose': 4693, 'number': 2084, 'color': 1832, 'shape': 561})
22423


In [9]:
img_guid2k = dict([(img_dataset[k]['Guid'], k) for k in img_dataset])
txt_guid2k = dict([(txt_dataset[k]['Guid'], k) for k in txt_dataset])
print(len(img_guid2k), len(txt_guid2k))

22423 24343


In [89]:
def create_pred_dataset(TH, output_dir, filename, guid2k, dataset):
    pred_dataset = {}
    count = 0
    filter_pred_json = json.load(open(os.path.join(output_dir, filename), "r"))
    for guid in filter_pred_json:
        num_choices = len(filter_pred_json[guid]['choices'])
        pred_scores = np.array(filter_pred_json[guid]['pred_scores'][:num_choices], dtype=float) # > th

        assert len(pred_scores) == num_choices

        pred_choices = [(filter_pred_json[guid]['choices'][i], pred_scores[i]) for i in range(num_choices) if pred_scores[i] >= TH]
        #pprint(pred_choices)
        pred_choices = sorted(pred_choices, key=lambda x: x[1], reverse=True)
        #pprint(pred_choices)
        try: k = guid2k[guid]
        except: k = guid
        pred_dataset[k] = copy.deepcopy(dataset[k])
        
        try: imgFacts = copy.deepcopy(dataset[k]['img_posFacts'] + dataset[k]['img_negFacts'])
        except: imgFacts = copy.deepcopy(dataset[k]['img_negFacts'])
        img_id2f = dict([(f['image_id'], copy.deepcopy(f)) for f in imgFacts])
        pred_dataset[k]['img_posFacts'] = []
        pred_dataset[k]['txt_posFacts'] = []
        for c in pred_choices:
            try: 
                image_id = str(int(c[0]))
                if len(image_id)<8: image_id = "0"*(8-len(image_id)) + image_id
                f = copy.deepcopy(img_id2f[image_id])
                f['filter_conf'] = c[1]
                pred_dataset[k]['img_posFacts'].append(f)
            except:
                pred_dataset[k]['txt_posFacts'].append({'fact': c[0], 'filter_conf': c[1]})
        count += 1
    print(count)
    print(len(pred_dataset))
    return pred_dataset


In [60]:
# TXT
output_dir = "/home/yingshac/CYS/WebQnA/VLP/vlp/light_output/filter_both_x_vinvl/"
filename = "test_-1_step3_txt_16_True_txt_dataset_0823_clean_te_UNknown_modality.json"
TH=0.25
pred_dataset = create_pred_dataset(TH, output_dir, filename, txt_guid2k, txt_dataset)
json.dump(pred_dataset, open(os.path.join(output_dir, "{}_th{}_{}".format("pred_dataset", str(th).split(".")[-1], filename)), "w"), indent=4)

4076
4076


In [90]:
# IMG
output_dir = "/home/yingshac/CYS/WebQnA/VLP/vlp/light_output/filter_both_x_vinvl/"
filename = "test_-1_step3_img_16_True_True_img_dataset_0823_clean_te_UNknown_modality.json"
TH=0.25
pred_dataset = create_pred_dataset(TH, output_dir, filename, img_guid2k, img_dataset)
json.dump(pred_dataset, open(os.path.join(output_dir, "{}_th{}_{}".format("pred_dataset", str(th).split(".")[-1], filename)), "w"), indent=4)

3464
3464


### Draft Zone

In [67]:
output_dir = "/home/yingshac/CYS/WebQnA/VLP/vlp/light_output/filter_both_x_vinvl/"
filename = "test_-1_step3_img_16_True_True_img_dataset_0823_clean_te_UNknown_modality.json"
tmp = json.load(open(os.path.join(output_dir, filename), "r"))

In [93]:
print(k)
k = random.choice(list(img_dataset.keys()))
pprint(img_dataset[k]['img_posFacts'])


2962
[{'caption': 'Taksim Square',
  'image_id': '00005887',
  'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/Taksim_Square.jpg/800px-Taksim_Square.jpg',
  'title': 'Taksim Square',
  'url': 'https://commons.wikimedia.org/wiki/File:Taksim_Square.jpg'}]


In [92]:
print(k)
k = random.choice(list(pred_dataset.keys()))
pprint(pred_dataset[k]['img_posFacts'])


18553
[{'caption': 'The Juilliard School photo D Ramey Logan',
  'filter_conf': 0.6538,
  'image_id': '00003462',
  'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/bd/The_Juilliard_School_photo_D_Ramey_Logan.jpg/757px-The_Juilliard_School_photo_D_Ramey_Logan.jpg',
  'title': 'The Juilliard School photo D Ramey Logan',
  'url': 'https://commons.wikimedia.org/wiki/File:The_Juilliard_School_photo_D_Ramey_Logan.jpg'},
 {'caption': 'UF Phillips Center',
  'filter_conf': 0.4924,
  'image_id': '00003206',
  'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/UF_Phillips_Center.JPG/800px-UF_Phillips_Center.JPG',
  'title': 'UF Phillips Center',
  'url': 'https://commons.wikimedia.org/wiki/File:UF_Phillips_Center.JPG'}]
